In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp1/2'
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    _dir = data_source1 + '/'
else:
    _dir = data_source1 + '\\'

clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]

In [32]:
#get cluster wise data for summarzation
count = 0
count_not = 0
count_yes = 0
precentage = 0
bellwether = {}
for cluster in clusters:
    df = pd.read_csv(cluster + '/1385_LR_bellwether_f1.csv')
    counts = []
    for row in range(df.shape[0]):
        j = df.iloc[row].values[1:]
        value = sum(i >= 0.66 for i in j)
        counts.append(value)
    df['>=0.6'] = counts
    precentage = max(counts)/df.shape[0]
    #print(cluster,df.iloc[counts.index(max(counts)),0])
    bellwether[cluster.rsplit('/',1)[1]] = df.iloc[counts.index(max(counts)),0]
    count += max(counts)


In [12]:
def prepare_data(path):
    df = pd.read_csv(path)
    df = df.drop(labels = ['Host','Vcs','Project','File','PL','IssueTracking'],axis=1)
    df = df.dropna()
    df = df[['TLOC', 'TNF', 'TNC', 'TND', 'LOC', 'CL', 'NStmt', 'NFunc',
       'RCC', 'MNL', 'avg_WMC', 'max_WMC', 'total_WMC', 'avg_DIT', 'max_DIT',
       'total_DIT', 'avg_RFC', 'max_RFC', 'total_RFC', 'avg_NOC', 'max_NOC',
       'total_NOC', 'avg_CBO', 'max_CBO', 'total_CBO', 'avg_DIT.1',
       'max_DIT.1', 'total_DIT.1', 'avg_NIV', 'max_NIV', 'total_NIV',
       'avg_NIM', 'max_NIM', 'total_NIM', 'avg_NOM', 'max_NOM', 'total_NOM',
       'avg_NPBM', 'max_NPBM', 'total_NPBM', 'avg_NPM', 'max_NPM', 'total_NPM',
       'avg_NPRM', 'max_NPRM', 'total_NPRM', 'avg_CC', 'max_CC', 'total_CC',
       'avg_FANIN', 'max_FANIN', 'total_FANIN', 'avg_FANOUT', 'max_FANOUT',
       'total_FANOUT', 'NRev', 'NFix', 'avg_AddedLOC', 'max_AddedLOC',
       'total_AddedLOC', 'avg_DeletedLOC', 'max_DeletedLOC',
       'total_DeletedLOC', 'avg_ModifiedLOC', 'max_ModifiedLOC',
       'total_ModifiedLOC','Buggy']]
    return df

def get_features(df):
    fs = feature_selector.featureSelector()
    df,_feature_nums,features = fs.cfs_bfs(df)
    return df,features

def apply_cfs(df):
    y = df.Buggy.values
    X = df.drop(labels = ['Buggy'],axis = 1)
    X = X.values
    selected_cols = CFS.cfs(X,y)
    cols = df.columns[[selected_cols]].tolist()
    cols.append('Buggy')
    return df[cols],cols
    
def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def load_data(path,target):
    df = pd.read_csv(path)
    if path == 'data/jm1.csv':
        df = df[~df.uniq_Op.str.contains("\?")]
    y = df[target]
    X = df.drop(labels = target, axis = 1)
    X = X.apply(pd.to_numeric)
    return X,y

# Cluster Driver
def cluster_driver(df,print_tree = True):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
    #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
    #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

In [13]:
attr_dict = pd.read_pickle('data/1385/projects/selected_attr.pkl')
attr_df = pd.DataFrame.from_dict(attr_dict,orient='index')
cluster,cluster_tree,max_depth = cluster_driver(attr_df)

[cluster_id=0] N_children: 9 N_samples: 697
> [cluster_id=1] N_children: 9 N_samples: 87
> > [cluster_id=2] N_children: 0 N_samples: 2
> > [cluster_id=3] N_children: 0 N_samples: 7
> > [cluster_id=4] N_children: 0 N_samples: 17
> > [cluster_id=5] N_children: 0 N_samples: 6
> > [cluster_id=6] N_children: 0 N_samples: 18
> > [cluster_id=7] N_children: 0 N_samples: 15
> > [cluster_id=8] N_children: 0 N_samples: 12
> > [cluster_id=9] N_children: 0 N_samples: 6
> > [cluster_id=10] N_children: 0 N_samples: 4
> [cluster_id=11] N_children: 2 N_samples: 4
> > [cluster_id=12] N_children: 0 N_samples: 1
> > [cluster_id=13] N_children: 0 N_samples: 3
> [cluster_id=14] N_children: 10 N_samples: 103
> > [cluster_id=15] N_children: 0 N_samples: 3
> > [cluster_id=16] N_children: 0 N_samples: 4
> > [cluster_id=17] N_children: 0 N_samples: 16
> > [cluster_id=18] N_children: 0 N_samples: 11
> > [cluster_id=19] N_children: 0 N_samples: 19
> > [cluster_id=20] N_children: 0 N_samples: 3
> > [cluster_id=21] 

In [28]:
_attr_dict = pd.read_pickle('data/1385/projects/other_selected_attr.pkl')
_attr_df = pd.DataFrame.from_dict(_attr_dict,orient='index')
test_projects = _attr_df.index.values.tolist()

In [24]:
predicted_cluster = cluster.predict(_attr_df,2)

In [37]:
for i in range(len(predicted_cluster)):
    c_id = predicted_cluster[i]
    s_project = bellwether[str(c_id)]
    s_path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted/' + s_project
    df = prepare_data(s_path)
    df.reset_index(drop=True,inplace=True)
    d = {'buggy': True, 'clean': False}
    df['Buggy'] = df['Buggy'].map(d)
    df, s_cols = apply_cfs(df)
    df = apply_smote(df)
    y = df.Buggy
    X = df.drop(labels = ['Buggy'],axis = 1)
    clf = LogisticRegression()
    clf.fit(X,y)
    
    d_project = test_projects[i]
    print(d_project)
    d_path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted/' + d_project
    _test_df = prepare_data(d_path)
    _df_test_loc = _test_df.LOC
    test_df = _test_df[s_cols]
    test_df.reset_index(drop=True,inplace=True)
    d = {'buggy': True, 'clean': False}
    test_df['Buggy'] = test_df['Buggy'].map(d)
    test_y = test_df.Buggy
    test_X = test_df.drop(labels = ['Buggy'],axis = 1)
    predicted = clf.predict(test_X)
    print(classification_report(test_y,predicted))

guineu.csv
              precision    recall  f1-score   support

       False       0.78      0.12      0.21        58
        True       0.32      0.92      0.48        26

    accuracy                           0.37        84
   macro avg       0.55      0.52      0.34        84
weighted avg       0.64      0.37      0.29        84

campsoft.csv
              precision    recall  f1-score   support

       False       0.00      0.00      0.00       104
        True       0.01      1.00      0.02         1

    accuracy                           0.01       105
   macro avg       0.00      0.50      0.01       105
weighted avg       0.00      0.01      0.00       105

gtad.csv
              precision    recall  f1-score   support

       False       0.00      0.00      0.00        71
        True       0.03      1.00      0.05         2

    accuracy                           0.03        73
   macro avg       0.01      0.50      0.03        73
weighted avg       0.00      0.03      0.